In [ ]:
from datetime import datetime
import pandas as pd
df = pd.read_csv('daylio_export.csv')


In [ ]:
# Daylio uses this awful format: a column with the year,
# a column with the month name in English and the day number (e.g. June 10), and another with the time
# So this ugly procedure is necessary
# moreover, this is also affected by locale
import locale
locale.setlocale(locale.LC_ALL, 'de_DE.UTF-8')
def to_date(r):
    
    datetime_object = datetime.strptime(r['date'].split()[0] + ' ' + r['date'].split()[1].zfill(2) + ' ' + str(r['year']) + ' ' + r['time'],
                                            '%d. %B %Y %H:%M')

    return datetime_object
df['new_date'] = df.apply(to_date, axis=1)
df = df.set_index('new_date')
df = df.drop(['year', 'date','weekday', 'time'], axis=1)

# use a number for the mood value, not the string
df['mood'] = df.apply(lambda x: ['Lausig','fugly','Ok','Gut','Super'].index(x['mood']),axis=1)
#df['mood'] = df.apply(lambda x: ['awful','fugly','meh','good','rad'].index(x['mood']),axis=1)

In [ ]:

%matplotlib inline
df['mood_smooth'] = df['mood'].rolling(window=10).mean()
df['mood_smooth'].plot(figsize=(10, 5))

In [ ]:
df['weekday'] = df.index.dayofweek
raw_dict = df.to_dict(orient='index')
# a dictionary with this key-value structure: 
# Timestamp('2016-07-07 22:03:00'): {'activities': 'work | friends',
#  'mood': 4,
#  'note': "something something"}, ...
from collections import Counter
import string
# dictionary mapping each punctuation codepoint to the space character one
punctuations_dict = dict.fromkeys(map(ord, string.punctuation), ord(' '))
words_general = Counter()
words_moods = [Counter() for i in range(5)]

for k,v in raw_dict.items():
    if type(v['note']) != str:
        # a record may not have a note
        print(f'ignored element for {k} {v}, had type {type(v["note"])}')
        continue
    # remove punctuations based on Unicode definition
    
    tokens =  v['note'].lower().translate(punctuations_dict).split()
    words_general.update(tokens)
    words_moods[v['mood']].update(tokens)

In [ ]:
print(len(words_general))
words_general.most_common(15)

Now we have the word frequencies for each mood and in general.

For each word we can calculate the ratio between the occurrences in some mood and the total occurrences, to get a measure of the correlation between the word and the mood.

Words appearing a few times or just once, however, would have an excessively high polarity (that is, the correlation with a specific category); we need to not give too much weight to them. This is called smoothing and is a common step in Bayes models. There are various methods, based in general on the idea of adding an "hidden" number of occurrences for each token/category combination, an approach stemming from the assumption that by sampling we ignored some values that would have appeared with a bigger (or infinite) sample size.

I'll use the simplest smoothing, called Laplacian smoothing: just add a value for each token/category combination. This way, the polarity of tokens appearing few times will be spread between all the categories, while more common tokens will have a more defined polarity.

In [ ]:
# for each token, add a fake sample to each mood pretending it appeared at least once for each
# if a mood actually had that token, it will just appear one more time
for w in words_general:
    for counter in words_moods:
        counter.update([w])
        

Now the polarity of the tokens for each category is calculated as the number of occurrences for that category divided by the total number of occurrences.

In [ ]:
polarities = {}
for w in words_general:
    # words_general was not smoothed, so it still contains the actual count
    total = words_general[w] + len(words_moods)
    polarities[w] = [counter[w]/total for counter in words_moods]
import heapq
for m in range(5):
    print(f'tokens with highest polarity with class {m}:')
    higher = heapq.nlargest(10,polarities.items(), key=lambda x:x[1][m])
    print(list(map(lambda k: k[0], higher)))